<a href="https://colab.research.google.com/github/kumar-abhishek/fsdl-talk-to-transformer/blob/master/notebooks/train_text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [69]:
# ! pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/master/examples/images/causal_language_modeling.png?raw=1)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](https://github.com/huggingface/notebooks/blob/master/examples/images/masked_language_modeling.png?raw=1)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

We will fine tune distilGPT-2 on the book Medidations by Marcus Aurelio for text generation

In [70]:
from datasets import load_dataset
# datasets = load_dataset('wikitext', 'wikitext-2-raw-v1') # uncomment this and comment next line if you would want to train on wikitext instead

In [71]:
!wget https://raw.githubusercontent.com/GITenberg/Meditations_2680/master/2680.txt -O text.txt
!wc -l text.txt
!head -88 text.txt
!cat text.txt | tail -7000 > train.txt
!cat text.txt | head -210 > val.txt
!head -20 train.txt
!rm -rf text.txt
!wc -l train.txt

--2021-05-13 13:55:07--  https://raw.githubusercontent.com/GITenberg/Meditations_2680/master/2680.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 425173 (415K) [text/plain]
Saving to: ‘text.txt’

text.txt            100%[===================>] 415.21K  --.-KB/s    in 0.1s    

2021-05-13 13:55:08 (3.83 MB/s) - ‘text.txt’ saved [425173/425173]

7212 text.txt
The Project Gutenberg EBook of Meditations, by Marcus Aurelius

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Meditations

Author: Marcus Aurelius

Posting Date: December 25, 2008 [EBook #2680]

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [72]:
datasets = load_dataset("text", data_files={"train": 'train.txt', "validation": 'val.txt'})

Using custom data configuration default-3fc3a25e90bd1008


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-3fc3a25e90bd1008/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [73]:
datasets["train"][10]


{'text': 'provinces were protected against oppression, and public help was given'}

In [74]:
datasets["validation"][10]

{'text': 'Author: Marcus Aurelius'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [75]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [76]:
show_random_elements(datasets["train"])

,text
0,
1,"to order all thy actions by that knowledge, doth daily waste and decay:"
2,"is implanted within him, a man must 'keep himself pure from all violent"
3,
4,"naturally sociable, doth little regard anything else: and above all"
5,This rule thou must remember to apply and make use of upon every conceit
6,"entire and perfect) is maimed, and mutilated, if thou shalt cut off"
7,"these two things divine, and things human. For without relation unto"
8,"death. Thus the empperor writes to his old friend, who had shown some"
9,"matter of great moment. And secondly, that that life which any the"


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

In [77]:
model_checkpoint = "distilgpt2"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [78]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [79]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [80]:
#tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text"])

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [81]:
tokenized_datasets["train"][1]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [46260,
  13,
  679,
  750,
  617,
  555,
  3083,
  1243,
  11,
  340,
  318,
  2081,
  13,
  1675,
  2251,
  257,
  9981,
  263]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [82]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [83]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [84]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    # num_proc=4,
)

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [85]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' place of father to the fatherless. Charitablefoundations were endowed for rearing and educating poor children. Theprovinces were protected against oppression, and public help was givento cities or districts which might be visited by calamity. The greatblot on his name, and one hard indeed to explain, is his treatmentof the Christians. In his reign Justin at Rome became a martyr tohis faith, and Polycarp at Smyrna, and we know of many outbreaks offanaticism in the provinces which caused the death of the faithful. Itis no excuse to plead that he knew nothing about the atrocities done in'

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [86]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

And some `TrainingArguments`:

In [87]:
from transformers import Trainer, TrainingArguments

In [88]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

We pass along all of those to the `Trainer` class:

In [89]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

And we can train our model:

In [90]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,4.222858


Epoch,Training Loss,Validation Loss
1,No log,4.222858
2,No log,4.187410
3,No log,4.182758


TrainOutput(global_step=279, training_loss=4.43495839983759, metrics={'train_runtime': 6311.9135, 'train_samples_per_second': 0.044, 'total_flos': 139468939001856.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 139612160, 'train_mem_cpu_peaked_delta': 0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [91]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 65.55


In [92]:
# genrate text now using the fine tuned model
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
torch.manual_seed(42)
# tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
# model = GPT2LMHeadModel.from_pretrained("distilgpt2", pad_token_id=tokenizer.eos_token_id)


def generate_text(string, possibilities=1):
    """ composition func
    Parameters
    ----------
    string : starting string
    Returns
    -------
    output : completed string
    """
    input = tokenizer.encode(string, return_tensors='pt')
    sampling_outputs = model.generate(
        input,
        do_sample=True,
        max_length=100,
        top_k=50,
        top_p=0.95,
        num_return_sequences=3)
    all_predictions = [tokenizer.decode(sample, skip_special_tokens=True) for sample in sampling_outputs]
    print("all_predictions: ")
    print(all_predictions)
    final_prediction = None
    # remove bad predictions:
    for prediction in all_predictions:
        if prediction is None or '' or prediction.endswith('\n\n\n\n\n') or prediction.count('.')>5:
            continue
        else:
            if '.' in prediction: # remove incomplete sentence in the end
                 prediction = prediction[:prediction.rfind('.')+1]
            final_prediction = prediction
            break

In [93]:
generate_text('J. Boulton')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


all_predictions: 
["J. Boulton: a number of the best physicians in the world. What are the differences among his physicians?I think the one thing which he was passionate about,was to have not one's opinions uponthe whole; but of such a common opinion, that there be many opinions in him concerning the whole of the world. Let his opinion be upon the whole, and to bear the same opinion, if only so that it is brought to the whole heart, and that it be disposed,", 'J. Boulton and the world: the world itself, which must and ought not be confused. Boulton, so much as a poet. D.A. is the best man he can get at any stage; and he can not only do things toil, and toil, and suffer, but also to the most painful, toil. He was so good and patient, and so much that a man ought to be the best man of all time; and by this which', 'J. Boulton was asked for his opinion. For his opinion, what he had done as an example of the man who was the man who had the pleasure of acting in the theatre, is that one of 

References:
1. https://colab.research.google.com/github/mrm8488/shared_colab_notebooks/blob/master/distilGPT2_finetuned_MA_Meditations.ipynb#scrollTo=n2Ui_GI2h-yF
2. https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/language_modeling.ipynb
3. https://github.com/huggingface/transformers/tree/master/examples/pytorch

In [97]:
trainer.save_model("distil-gpt2-meditations-trained-model")
tokenizer.save_pretrained("distil-gpt2-meditations-tokenizer")

('distil-gpt2-meditations-tokenizer/tokenizer_config.json',
 'distil-gpt2-meditations-tokenizer/special_tokens_map.json',
 'distil-gpt2-meditations-tokenizer/vocab.json',
 'distil-gpt2-meditations-tokenizer/merges.txt',
 'distil-gpt2-meditations-tokenizer/added_tokens.json',
 'distil-gpt2-meditations-tokenizer/tokenizer.json')